In [4]:
##IMPORTED THE LIBRARIES 
import pandas as pd
import numpy as np
from collections import Counter

In [5]:
##READ DATASETS
b2b_df = pd.read_csv("data/B2B_Partner.csv")
complaints_df = pd.read_csv("data/complaints.csv")
player_df = pd.read_csv("data/Player_data.csv")
billing_df = pd.read_csv("data/Billing.csv")
contracts_df = pd.read_csv("data/contracts.csv")
users_df = pd.read_csv("data/User_subscription.csv")

## JOINS OR MERGE 


In [6]:
player_df.head()

,EventType,SessionStart,Timezone,Subscribed,Platform,PartnerId,ApplicationVersion,Location,Hardware,operatingSystem,...,DeliveryProtocol,Playing_rate,StartingBitrate,Video_resolution,Start_position,End_position,UserId,DeviceId,Video_position,Duration
0,pause,2021-03-28 12:54:12,Africa/Douala,True,FireTV,hotstar,2.7,Bhimavaram,FireTV,Android,...,Cloudfront,NaN,7.6,2K,00:25,00:57,jeffreylewis,407ta-N84G2-9o5LH-DP8Qe,00:56,1848.0
1,stop,2021-01-02 04:12:17,America/Montevideo,True,Browser,hotstar,2.5,Phagwara,PC/Laptop,MacOS,...,Cloudfront,NaN,5.5,480p,00:29,00:41,jessica41,856cp-i25o9-9u3YO-JL8YY,00:34,307.0
2,play,2021-07-06 17:12:01,Africa/Harare,False,Android,hotstar,4.9,Ranchi,Android Phone,Android,...,Cloudfront,NaN,8.7,720p,00:26,00:48,justin83,962gJ-n52j8-8F9Jl-Gy3UV,00:38,730.0
3,skip,2021-09-29 23:07:07,America/Nassau,False,FireTV,hotstar,4.0,Vadodara,FireTV,Android,...,Akamai,NaN,9.4,480p,01:22,02:00,michael21,347Vt-F15u7-6W3zh-Ki8BY,01:59,2188.0
4,play,2021-05-10 01:51:44,Africa/Blantyre,False,iOS,hotstar,1.9,Gandhinagar,iPhone,iOS,...,Akamai,NaN,9.9,720p,00:42,02:35,parker98,464Mg-D08Q1-6B0iD-ve1mH,01:36,3239.0


In [7]:
users_df.head()

,profile_ID,Contact_type,DOB,Email,First_Name,Gender,Mobile_Number,Pack_ID,Start_Date,Status,Upgradable,Validity_End,Country,State,customerID,Zipcode
0,rsanchez,Email,6/10/2005,ldixson3h@engadget.com,Leese,Female,391-447-5117,432281,10/28/2021,INACTIVE,YES,12/5/2022,India,Goa,2546-KZAAT,665 33
1,karina10,Email,1/12/2005,rearngy40@sphinn.com,Rubetta,Female,777-570-1291,432300,2/10/2021,INACTIVE,NO,11/22/2022,India,Goa,2371-KFUOG,111721
2,sethconrad,Email,10/30/1992,tbecom72@spotify.com,Tait,Male,901-892-5768,432410,10/30/2022,ACTIVE,YES,11/10/2022,India,Goa,0188-GWFLE,NaN
3,ewilson,Email,8/14/2015,elavalde9g@google.ru,Eugenia,Female,711-627-9420,432496,1/14/2022,ACTIVE,NO,12/27/2022,India,Goa,0289-IVARM,NaN
4,evanvelasquez,Email,5/25/1992,dmencib1@cisco.com,Devan,Female,902-541-3157,432553,1/21/2022,INACTIVE,YES,11/9/2022,India,Goa,7672-VFMXZ,NaN


In [7]:
player_df.to_csv("player_df.csv", index = False)

In [8]:
def get_most_common_dev(devlist):
    c = Counter(devlist)
    return c.most_common(1)[0][0]

In [9]:
agg_dict = {
    "Duration":"mean",
    "DeviceId": pd.Series.nunique,
    "Hardware": get_most_common_dev,
    
}


player_agged = player_df.groupby("UserId").agg(agg_dict)

In [10]:
player_agged.reset_index(inplace = True)

In [11]:
player_agged.rename(columns = {"Duration":"avg_dur", 
                               "DeviceId":"num_devs", 
                               "Hardware":"freq_used_dev"}, inplace = True)

In [12]:
users_players_joined = users_df.merge(player_agged, left_on="profile_ID", right_on = "UserId", how = "left")

In [13]:
agg_dict = {
    "partner id":pd.Series.nunique,
    "device id": pd.Series.nunique,
    "device type": get_most_common_dev,
    
}


b2b_agged = b2b_df.groupby("customerID").agg(agg_dict)

In [14]:
b2b_agged.reset_index(inplace = True)

In [15]:
b2b_agged.rename(columns = {"partner id":"part_id", 
                            "deviceId":"num_devs", 
                            "device type":"freq_used_type"}, inplace = True)

In [16]:
users_b2b_joined = users_df.merge(b2b_agged, on = "customerID", how = "left")

In [17]:
users_b2b_joined

,profile_ID,Contact_type,DOB,Email,First_Name,Gender,Mobile_Number,Pack_ID,Start_Date,Status,Upgradable,Validity_End,Country,State,customerID,Zipcode,part_id,device id,freq_used_type
0,rsanchez,Email,6/10/2005,ldixson3h@engadget.com,Leese,Female,391-447-5117,432281,10/28/2021,INACTIVE,YES,12/5/2022,India,Goa,2546-KZAAT,665 33,1,1,tv
1,karina10,Email,1/12/2005,rearngy40@sphinn.com,Rubetta,Female,777-570-1291,432300,2/10/2021,INACTIVE,NO,11/22/2022,India,Goa,2371-KFUOG,111721,1,1,mobile
2,sethconrad,Email,10/30/1992,tbecom72@spotify.com,Tait,Male,901-892-5768,432410,10/30/2022,ACTIVE,YES,11/10/2022,India,Goa,0188-GWFLE,NaN,1,1,tv
3,ewilson,Email,8/14/2015,elavalde9g@google.ru,Eugenia,Female,711-627-9420,432496,1/14/2022,ACTIVE,NO,12/27/2022,India,Goa,0289-IVARM,NaN,1,1,mobile
4,evanvelasquez,Email,5/25/1992,dmencib1@cisco.com,Devan,Female,902-541-3157,432553,1/21/2022,INACTIVE,YES,11/9/2022,India,Goa,7672-VFMXZ,NaN,1,1,mobile
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,brandili,WhatsApp,12/14/2021,mcokee0@psu.edu,Maryjane,Female,755-848-1351,432660,3/22/2022,INACTIVE,NO,12/26/2022,India,Arunachal Pradesh,7361-YPXFS,NaN,1,1,mobile
996,davidlowe,WhatsApp,4/1/1993,ctothex@google.com.hk,Chic,Male,472-131-0900,432693,9/25/2021,INACTIVE,YES,11/14/2022,India,Arunachal Pradesh,3466-BYAVD,NaN,1,1,mobile
997,carrierivera,WhatsApp,9/30/2002,bmacevillyh3@prlog.org,Bili,Female,568-875-8493,432771,11/24/2021,INACTIVE,YES,12/3/2022,India,Arunachal Pradesh,5652-MSDEY,6340,1,1,mobile
998,ronniecoffey,WhatsApp,8/29/2002,lberrowhy@smh.com.au,Lilli,Female,380-420-0034,432802,1/21/2022,ACTIVE,NO,12/16/2022,India,Arunachal Pradesh,9970-QBCDA,NaN,1,1,laptop


In [18]:
users_players_joined

,profile_ID,Contact_type,DOB,Email,First_Name,Gender,Mobile_Number,Pack_ID,Start_Date,Status,Upgradable,Validity_End,Country,State,customerID,Zipcode,UserId,avg_dur,num_devs,freq_used_dev
0,rsanchez,Email,6/10/2005,ldixson3h@engadget.com,Leese,Female,391-447-5117,432281,10/28/2021,INACTIVE,YES,12/5/2022,India,Goa,2546-KZAAT,665 33,rsanchez,1512.277778,18,Android Phone
1,karina10,Email,1/12/2005,rearngy40@sphinn.com,Rubetta,Female,777-570-1291,432300,2/10/2021,INACTIVE,NO,11/22/2022,India,Goa,2371-KFUOG,111721,karina10,1329.500000,8,iPad
2,sethconrad,Email,10/30/1992,tbecom72@spotify.com,Tait,Male,901-892-5768,432410,10/30/2022,ACTIVE,YES,11/10/2022,India,Goa,0188-GWFLE,NaN,sethconrad,1478.166667,18,PC/Laptop
3,ewilson,Email,8/14/2015,elavalde9g@google.ru,Eugenia,Female,711-627-9420,432496,1/14/2022,ACTIVE,NO,12/27/2022,India,Goa,0289-IVARM,NaN,ewilson,1118.133333,15,PC/Laptop
4,evanvelasquez,Email,5/25/1992,dmencib1@cisco.com,Devan,Female,902-541-3157,432553,1/21/2022,INACTIVE,YES,11/9/2022,India,Goa,7672-VFMXZ,NaN,evanvelasquez,1557.428571,21,FireTV
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,brandili,WhatsApp,12/14/2021,mcokee0@psu.edu,Maryjane,Female,755-848-1351,432660,3/22/2022,INACTIVE,NO,12/26/2022,India,Arunachal Pradesh,7361-YPXFS,NaN,brandili,1266.307692,13,PC/Laptop
996,davidlowe,WhatsApp,4/1/1993,ctothex@google.com.hk,Chic,Male,472-131-0900,432693,9/25/2021,INACTIVE,YES,11/14/2022,India,Arunachal Pradesh,3466-BYAVD,NaN,davidlowe,1199.000000,18,FireTV
997,carrierivera,WhatsApp,9/30/2002,bmacevillyh3@prlog.org,Bili,Female,568-875-8493,432771,11/24/2021,INACTIVE,YES,12/3/2022,India,Arunachal Pradesh,5652-MSDEY,6340,carrierivera,1482.000000,17,FireTV
998,ronniecoffey,WhatsApp,8/29/2002,lberrowhy@smh.com.au,Lilli,Female,380-420-0034,432802,1/21/2022,ACTIVE,NO,12/16/2022,India,Arunachal Pradesh,9970-QBCDA,NaN,ronniecoffey,1518.000000,30,PC/Laptop


In [19]:
billing_df 

,Customer Id,Billing Id,Subscription Type,Billing Type,Customer Longevity
0,1469590,432156,12 Months,Paytm,4 Years
1,1203873,432157,3 Months,NetBanking,2 Years
2,1432761,432158,6 Months,NetBanking,3 Years
3,1207582,432159,12 Months,Paytm,4 Years
4,1674436,432160,12 Months,Paytm,4 Years
...,...,...,...,...,...
995,1880194,433151,12 Months,Paytm,4 Years
996,1114064,433152,12 Months,Paytm,4 Years
997,1758046,433153,12 Months,Paytm,4 Years
998,1824545,433154,12 Months,Paytm,4 Years


In [20]:
billing_df["Customer Id"] = users_players_joined["customerID"]

In [21]:
users_training = users_players_joined.merge(billing_df, left_on = 'customerID', right_on = "Customer Id", how = 'left')

In [22]:
complaints_df.columns

Index(['IssueType', 'deviceType', 'IssueTime', 'IssueDetails', 'mobileNumber',
       'emailId', 'mode', 'cdn', 'userIdentifier', 'deviceDetails',
       'OSDetails', 'platform', 'appVersion', 'country', 'state',
       'networkStrength', 'contentID', 'Advertise /  Video'],
      dtype='object')

In [23]:
users_training.columns
users_training

,profile_ID,Contact_type,DOB,Email,First_Name,Gender,Mobile_Number,Pack_ID,Start_Date,Status,...,Zipcode,UserId,avg_dur,num_devs,freq_used_dev,Customer Id,Billing Id,Subscription Type,Billing Type,Customer Longevity
0,rsanchez,Email,6/10/2005,ldixson3h@engadget.com,Leese,Female,391-447-5117,432281,10/28/2021,INACTIVE,...,665 33,rsanchez,1512.277778,18,Android Phone,2546-KZAAT,432156,12 Months,Paytm,4 Years
1,karina10,Email,1/12/2005,rearngy40@sphinn.com,Rubetta,Female,777-570-1291,432300,2/10/2021,INACTIVE,...,111721,karina10,1329.500000,8,iPad,2371-KFUOG,432157,3 Months,NetBanking,2 Years
2,sethconrad,Email,10/30/1992,tbecom72@spotify.com,Tait,Male,901-892-5768,432410,10/30/2022,ACTIVE,...,NaN,sethconrad,1478.166667,18,PC/Laptop,0188-GWFLE,432158,6 Months,NetBanking,3 Years
3,ewilson,Email,8/14/2015,elavalde9g@google.ru,Eugenia,Female,711-627-9420,432496,1/14/2022,ACTIVE,...,NaN,ewilson,1118.133333,15,PC/Laptop,0289-IVARM,432159,12 Months,Paytm,4 Years
4,evanvelasquez,Email,5/25/1992,dmencib1@cisco.com,Devan,Female,902-541-3157,432553,1/21/2022,INACTIVE,...,NaN,evanvelasquez,1557.428571,21,FireTV,7672-VFMXZ,432160,12 Months,Paytm,4 Years
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,brandili,WhatsApp,12/14/2021,mcokee0@psu.edu,Maryjane,Female,755-848-1351,432660,3/22/2022,INACTIVE,...,NaN,brandili,1266.307692,13,PC/Laptop,7361-YPXFS,433151,12 Months,Paytm,4 Years
996,davidlowe,WhatsApp,4/1/1993,ctothex@google.com.hk,Chic,Male,472-131-0900,432693,9/25/2021,INACTIVE,...,NaN,davidlowe,1199.000000,18,FireTV,3466-BYAVD,433152,12 Months,Paytm,4 Years
997,carrierivera,WhatsApp,9/30/2002,bmacevillyh3@prlog.org,Bili,Female,568-875-8493,432771,11/24/2021,INACTIVE,...,6340,carrierivera,1482.000000,17,FireTV,5652-MSDEY,433153,12 Months,Paytm,4 Years
998,ronniecoffey,WhatsApp,8/29/2002,lberrowhy@smh.com.au,Lilli,Female,380-420-0034,432802,1/21/2022,ACTIVE,...,NaN,ronniecoffey,1518.000000,30,PC/Laptop,9970-QBCDA,433154,12 Months,Paytm,4 Years


In [24]:
complaints_df["deviceType"] = users_training["freq_used_dev"]

In [25]:
users_complaints_training = users_training.merge(complaints_df, left_on = 'freq_used_dev', right_on = "deviceType", how = 'left')

users_complaints_training.


In [26]:
users_complaints_training.columns

Index(['profile_ID', 'Contact_type', 'DOB', 'Email', 'First_Name', 'Gender',
       'Mobile_Number', 'Pack_ID', 'Start_Date', 'Status', 'Upgradable',
       'Validity_End', 'Country', 'State', 'customerID', 'Zipcode', 'UserId',
       'avg_dur', 'num_devs', 'freq_used_dev', 'Customer Id', 'Billing Id',
       'Subscription Type ', 'Billing Type ', 'Customer Longevity',
       'IssueType', 'deviceType', 'IssueTime', 'IssueDetails', 'mobileNumber',
       'emailId', 'mode', 'cdn', 'userIdentifier', 'deviceDetails',
       'OSDetails', 'platform', 'appVersion', 'country', 'state',
       'networkStrength', 'contentID', 'Advertise /  Video'],
      dtype='object')

In [27]:
contracts_df

,Unnamed: 0,Contract Id,Contract Description,Contract Amount,Contract Effective Date,Contract End Date,Currency Code,Contract Term,SLA Scheme Id,Customer Segment
0,0,32-8540983,sit amet justo morbi ut odio cras mi pede male...,1500,2019-11-22,2020-08-31,INR,12 Months,#26db3a,Empty
1,1,25-9229010,est lacinia nisi venenatis tristique fusce con...,650,2019-08-26,2020-06-18,INR,3 Months,#0344fe,Empty
2,2,78-7194379,vestibulum ante ipsum primis in faucibus orci ...,350,2021-05-02,2021-08-02,INR,1 Month,#b3409b,Empty
3,3,76-2104357,platea dictumst morbi vestibulum velit id pret...,350,2020-11-27,2023-08-30,INR,1 Month,#a6dcb3,Empty
4,4,98-2263265,feugiat et eros vestibulum ac est lacinia nisi...,650,2021-10-27,2022-07-24,INR,3 Months,#5dcc6e,Empty
...,...,...,...,...,...,...,...,...,...,...
995,995,71-0228805,nisi eu orci mauris lacinia sapien quis libero...,1500,2022-04-12,2021-08-22,INR,12 Months,#c0378c,Empty
996,996,15-9310366,posuere cubilia curae nulla dapibus dolor vel ...,650,2020-06-09,2020-08-19,INR,3 Months,#93853a,Empty
997,997,49-1354785,cubilia curae donec pharetra magna vestibulum ...,350,2020-06-21,2022-01-02,INR,1 Month,#e0b4f8,Empty
998,998,71-0828876,quam sapien varius ut blandit non interdum in ...,1500,2019-05-09,2021-01-01,INR,12 Months,#ffacbc,Empty


In [30]:
users_complaints_training["IssueDetails"] = contracts_df["Contract Description"]


In [31]:
users_contracts = users_complaints_training.merge(contracts_df, left_on="IssueDetails", right_on = "Contract Description", how = "left")

In [32]:
users_contracts.columns

Index(['profile_ID', 'Contact_type', 'DOB', 'Email', 'First_Name', 'Gender',
       'Mobile_Number', 'Pack_ID', 'Start_Date', 'Status', 'Upgradable',
       'Validity_End', 'Country', 'State', 'customerID', 'Zipcode', 'UserId',
       'avg_dur', 'num_devs', 'freq_used_dev', 'Customer Id', 'Billing Id',
       'Subscription Type ', 'Billing Type ', 'Customer Longevity',
       'IssueType', 'deviceType', 'IssueTime', 'IssueDetails', 'mobileNumber',
       'emailId', 'mode', 'cdn', 'userIdentifier', 'deviceDetails',
       'OSDetails', 'platform', 'appVersion', 'country', 'state',
       'networkStrength', 'contentID', 'Advertise /  Video', 'Unnamed: 0',
       ' Contract Id', 'Contract Description', 'Contract Amount',
       'Contract Effective Date', 'Contract End Date', 'Currency Code',
       'Contract Term', 'SLA Scheme Id', 'Customer Segment'],
      dtype='object')

In [33]:
colums_needed = [
    'profile_ID',
    'DOB',
    'Contact_type',
    'Gender',
    'Pack_ID',
    'Start_Date',
    'Status',
    'Upgradable',
    'Validity_End',
    'Country',
    'State',
    'avg_dur', 
    'num_devs', 
    'freq_used_dev',
    'Subscription Type ', 
    'Billing Type ',
    'Customer Longevity',
    'Advertise /  Video',
    'networkStrength',
    'appVersion',
    'platform',
    'cdn',
    'Contract Term',
    'Contract Amount'
    
]

# TRAINING DATA

In [34]:
train_df = users_contracts[colums_needed]
train_df.columns = [c.strip() for c in train_df.columns]

In [35]:
train_df["Subscription Type"] = train_df["Subscription Type"].str.replace(" Months", "")

/home/kezia/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [36]:
train_df["Customer Longevity"] = train_df["Customer Longevity"].str.replace(" Years", "")

/home/kezia/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [37]:
train_df

,profile_ID,DOB,Contact_type,Gender,Pack_ID,Start_Date,Status,Upgradable,Validity_End,Country,...,Subscription Type,Billing Type,Customer Longevity,Advertise / Video,networkStrength,appVersion,platform,cdn,Contract Term,Contract Amount
0,rsanchez,6/10/2005,Email,Female,432281,10/28/2021,INACTIVE,YES,12/5/2022,India,...,12,Paytm,4,Video,,macOS Ventura 13.0.1,YouTube TV,Cloudfront,12 Months,1500.0
1,rsanchez,6/10/2005,Email,Female,432281,10/28/2021,INACTIVE,YES,12/5/2022,India,...,12,Paytm,4,Video,,Windows 10,Disney + Hotstar,Cloudfront,3 Months,650.0
2,rsanchez,6/10/2005,Email,Female,432281,10/28/2021,INACTIVE,YES,12/5/2022,India,...,12,Paytm,4,Video,,macOS Ventura 13.0.1,Sonyliv,Akamai,1 Month,350.0
3,rsanchez,6/10/2005,Email,Female,432281,10/28/2021,INACTIVE,YES,12/5/2022,India,...,12,Paytm,4,Advertise,,Xcode 14.1,Sling TV,Cloudfront,1 Month,350.0
4,rsanchez,6/10/2005,Email,Female,432281,10/28/2021,INACTIVE,YES,12/5/2022,India,...,12,Paytm,4,Video,,Android TV 12.3.2,Sonyliv,Cloudfront,3 Months,650.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
332119,kathleen63,5/28/1989,WhatsApp,Genderqueer,433001,7/24/2021,ACTIVE,NO,12/11/2022,India,...,12,Paytm,4,Advertise,,macOS Ventura 13.0.1,Facebook Watch,Cloudfront,NaN,NaN
332120,kathleen63,5/28/1989,WhatsApp,Genderqueer,433001,7/24/2021,ACTIVE,NO,12/11/2022,India,...,12,Paytm,4,Video,,Xcode 14.1,HBO Max,Akamai,NaN,NaN
332121,kathleen63,5/28/1989,WhatsApp,Genderqueer,433001,7/24/2021,ACTIVE,NO,12/11/2022,India,...,12,Paytm,4,Video,,None,Jio Tv,Cloudfront,NaN,NaN
332122,kathleen63,5/28/1989,WhatsApp,Genderqueer,433001,7/24/2021,ACTIVE,NO,12/11/2022,India,...,12,Paytm,4,Advertise,,FireOS 7.6.1.3,Disney + Hotstar,Cloudfront,NaN,NaN


In [38]:
train_df.to_csv("data/training_data(new).csv", index = False)

In [39]:
churn_df = pd.DataFrame()
churn_df["profile_ID"] = train_df["profile_ID"]
churn_df["churn"] = np.random.choice([0,1], size = len(churn_df), p = [0.8,0.2])

In [40]:
churn_df.to_csv("data/churn.csv", index = False)